시간, 기후, 계절, 요일 정보를 병합하기
2020 06 01 ~ 2022 06 28 데이터(+ 2022 07 01 ~ 2022 08 31 데이터) 일단 앞 데이터로만

년|월|일|요일|공휴일 유무|온도|습도|건물이름 유효전력량*56

['0_SV-2', '1_SV-5', '2_SV-6', '3_SV-7', '4_HV-NM1', '5_HV-NM2', '6_고압콘덴샤', '7_신재생에너지동', '8_대학B동', '9_대학기숙사A동', '10_제2학생회관', '11_학사P/P', '12_교원아파트', '13_대학C동', '14_중앙연구기기센터', '15_대학A동', '16_다산빌딩', '17_산학협력연구동', '18_신재생에너지동(E)', '19_대학B동(E)', '20_대학기숙사A동(E)', '21_학사P/P(E)', '22_제2학생회관(E)', '23_교원아파트(E)', '24_대학C동(E)', '25_중앙연구기기센터(E)', '26_대학A동(E)', '27_다산빌딩(E)', '28_산학협력연구동(E)', '29_SV-2', '30_SV-5', '31_SV-6', '32_HV-M1', '33_HV-M2', '34_HV-NM', '35_HV-EM', '36_중앙P/P', '37_전기전자컴퓨터공학동', '38_신소재공학동', '39_생명과학동', '40_기계공학동', '41_LG도서관', '42_창업B동', '43_금호관', '44_냉각수순환펌프1,2,3', '45_냉각수순환펌프4,5,6', '46_기혼자아파트E동', '47_기숙사9동', '48_고등광/극초단', '49_신소재공학동(E)', '50_전기전자컴퓨터공학동(E)', '51_생명과학동(E)', '52_기계공학동(E)', '53_LG도서관(E)', '54_중앙P/P(E)', '55_고등광연구소(E)']

memo
학사 - 빌딩 29개 - MWH 27개 - KWH 2개(산학협력연구동)
석사 - 빌딩 27개 - MWH 27개

In [136]:
import os
import pandas as pd
from datetime import timedelta, date, datetime
from workalendar.asia import SouthKorea

def get_building(df):
    name = []
    temp = df[df['전력량 일보'] == '구분'] # 엑셀에서 '전력량 일보'라 써진 열에서 '구분' 행만 읽기
    temp = temp.values.tolist()
    for i in temp[0]:
        if isinstance(i, str) and i != '구분':
            name.append(i)
    # print(name)
    return name

def get_all_building(file1, file2):
    df1 = pd.read_excel(file1)
    df1 = df1.dropna(how='all')
    df2 = pd.read_excel(file2)
    df2 = df2.dropna(how='all')
    buildings = []
    index = 0
    for i in get_building(df1):
        buildings.append(str(index)+'_'+i)
        index += 1
    for i in get_building(df2):
        buildings.append(str(index)+'_'+i)
        index += 1
    # print(buildings)
    return buildings

def get_MWH(df):
    time = df.filter(like='시간')
    mwh_df = df.filter(regex='MWH|KWH') # 유효전력량 단위 다른 거 처리하기 17&28 건물
    return time, mwh_df

def get_all_MWH(file1, file2, bs):
    df1 = pd.read_excel(file1, header=9)
    df1 = df1.dropna(how='all')
    df2 = pd.read_excel(file2, header=9)
    df2 = df2.dropna(how='all')

    time, mwh_df1 = get_MWH(df1)
    _, mwh_df2 =  get_MWH(df2)
    df = pd.concat([time, mwh_df1, mwh_df2], axis=1)
    bs.insert(0, '')
    df.columns = bs
    df['17_산학협력연구동'] = df['17_산학협력연구동'] / 1000
    df['28_산학협력연구동(E)'] = df['28_산학협력연구동(E)'] / 1000
    return df

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)+1):
        yield start_date + timedelta(n)

def get_holiday(year):
    cal = SouthKorea()
    holidays = cal.holidays(year)  # 2022년의 공휴일을 가져옵니다.
    holidays = [holiday[0] for holiday in holidays]
    holidays = [datetime.strptime(str(holiday), '%Y-%m-%d').date() for holiday in holidays] # 날짜 형식으로 변환
    return holidays

In [97]:
# example
# folder = '/home/kimyirum/EMS/data/2023_DATA/load/'
# save = '/home/kimyirum/EMS/data/dataset_example/'
folder = '/home/kimyirum/EMS/ict-2023-ems/load/data/2023_DATA/load/'
save = '/home/kimyirum/EMS/ict-2023-ems/load/data/dataset_example/'

day = '2022-08-31'
file1 = folder + '학사 일보.gcf_' + day + '_23-59.xls'
file2 = folder + '석사 일보.gcf_' + day + '_23-59.xls'

bs = get_all_building(file1, file2)
data = get_all_MWH(file1, file2, bs)
# print(data.head())
print(data.shape)

data.to_excel(save+day+'.xlsx', index=False)

(24, 57)


In [104]:
# Generate
# folder = '/home/kimyirum/EMS/data/2023_DATA/load/'
# save = '/home/kimyirum/EMS/data/dataset/'
folder = '/home/kimyirum/EMS/ict-2023-ems/load/data/2023_DATA/load/'
save = '/home/kimyirum/EMS/ict-2023-ems/load/data/dataset/'

################################
start_date = date(2022, 7, 1)
end_date = date(2022, 8, 31)
################################

date_list = [single_date.strftime("%Y-%m-%d") for single_date in daterange(start_date, end_date)]
for d in date_list:
    file1 = folder + '학사 일보.gcf_' + d + '_23-59.xls'
    file2 = folder + '석사 일보.gcf_' + d + '_23-59.xls'
    bs = get_all_building(file1, file2)
    data = get_all_MWH(file1, file2, bs)
    data.to_excel(save+d+'.xlsx', index=False)

print("Save "+str(len(os.listdir(save)))+" files")

Save 62 files


In [137]:
# Generate
# power_data_folder = '/home/kimyirum/EMS/data/dataset/'
# weather_data_file = "/home/kimyirum/EMS/data/2023_DATA/weather.xls"
# save_file = "/home/kimyirum/EMS/data/merged_data.xlsx"
power_data_folder = '/home/kimyirum/EMS/ict-2023-ems/load/data/dataset/'
weather_data_file = "/home/kimyirum/EMS/ict-2023-ems/load/data/2023_DATA/weather.xls"
save_file = "/home/kimyirum/EMS/ict-2023-ems/load/data/merged_data.xlsx"

# 폴더 내의 모든 파일을 읽어들이며 병합
power_dfs = []
for file_name in os.listdir(power_data_folder):
    file_path = os.path.join(power_data_folder, file_name)
    df = pd.read_excel(file_path)
    # 날짜 정보를 파일 이름에서 추출하거나 파일 내용에서 추출하여 추가
    df['date'] = pd.to_datetime(file_name.replace('.xlsx', ''))
    power_dfs.append(df)

# 모든 데이터프레임을 병합
power_df = pd.concat(power_dfs)
power_df = power_df.rename(columns={'Unnamed: 0':'time'})
power_df['time'] = power_df['time'].str.replace('시', '').astype(int)
power_df['date'] = power_df['date'] + pd.to_timedelta(power_df['time'], unit='h')
header_list = power_df.columns.tolist()

# 날씨 데이터 읽기
weather_df = pd.read_excel(weather_data_file)
weather_df['date'] = pd.to_datetime(weather_df['날짜'])

# print(power_df)
# print(weather_df)

# 날씨 데이터와 전력 사용량 데이터 병합
merged_df = pd.merge(weather_df, power_df, on='date')

# '년', '월', '일', '요일' 열 추가
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month
merged_df['day'] = merged_df['date'].dt.day
merged_df['dayofweek'] = merged_df['date'].dt.dayofweek+1  # 월요일: 1, 일요일: 7
merged_df['holiday'] = merged_df['date'].apply(lambda x: 1 if x.date() in get_holiday(2022) else 0)

header_list.pop(0)
header_list.pop(-1)
new_order = ['month', 'day', 'time', 'dayofweek', 'holiday', '온도', '습도'] + header_list
final_df = merged_df.reindex(columns=new_order)
final_df = final_df.rename(columns={'온도': 'temperature', '습도': 'humidity'})

# 최종 데이터프레임을 엑셀 파일로 저장
final_df.to_excel(save_file, index=False)
print(final_df.shape) # 62일*24시간 = 1488, 7개 입력변수+건물56개 = 63

(1488, 63)
